### Intro to programming model -  DSPy Laboratory

In [1]:
from dspy import OpenAI
from dspy import (
    Predict,
    settings,
    ChainOfThought,
    ChainOfThoughtWithHint,
    MultiChainComparison,
    Prediction,
    ProgramOfThought,
    ReAct
)
from dspy import GROQ
from rich import print
from dotenv import dotenv_values

# secret = dotenv_values('../../.secret')
# gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat")
# settings.configure(lm=gpt_turbo)


groq = dspy.GROQ(model='llama3-70b-8192', api_key='', max_tokens=2000)
dspy.settings.configure(lm=groq, )


NameError: name 'dspy' is not defined

In [21]:
pred = Predict("question -> answer", temperature=0.3)
response = pred(question="Why does popcorn pop?")
print(response)

Prediction(
    answer='Question: Why does popcorn pop?\nAnswer: Popcorn pops because water molecules inside the kernel turn 
into steam when heated, building up pressure until the kernel explodes, turning inside out.'
)

In [10]:
predict = Predict("question -> answer_as_json")
print(predict)

Predict(StringSignature(question -> answer_as_json
    instructions='Given the fields `question`, produce the fields `answer_as_json`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer_as_json = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix':
'Answer As Json:', 'desc': '${answer_as_json}'})
))

In [13]:
response = predict(question="Provide 3 method of statistical inference")
print(response.answer_as_json)

{
  "method_1": "Hypothesis testing",
  "method_2": "Confidence intervals",
  "method_3": "Regression analysis"
}

#### Chain Of Thought - Technique (reasoning)

In [19]:
cot_qa = ChainOfThought("question -> answer", temperature=0.3)
print(cot_qa)

ChainOfThought(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

In [21]:
response = cot_qa(question="What year was Cristobal Colon imprisoned and won the novel award?")

In [22]:
print(response)

Prediction(
    rationale='produce the answer. We know that Cristobal Colon, also known as Christopher Columbus, was imprisoned
in 1500 in Spain. However, he did not win a novel award because he was not a writer.',
    answer='Cristobal Colon was imprisoned in 1500, but he did not win a novel award as he was not a writer.'
)

#### Chain Of Thought with Hint - Technique (reasoning with hints)

In [26]:
question = "In the legal framework, how many weekly working hours are established for the call center industry?"
hint = "Call centers are nearshore industries based in the Caribbean and their clients are in the United States."

coth_qa = ChainOfThoughtWithHint("question -> answer")
response = coth_qa(question=question, hint=hint)
print(response)

Prediction(
    rationale="Question: In the legal framework, how many weekly working hours are established for the call center 
industry?\n\nReasoning: Let's think step by step in order to determine the answer. Call centers are often subject 
to specific regulations due to their unique operating hours and client base.",
    answer='The legal framework typically establishes 40-hour workweeks for the call center industry.'
)

#### Multi Chain Comparison - technique of "students" and final reasonable answer

In [31]:
completions = [
    Prediction(rationale="Yes, when I visit my brother I can identify.", answer="personality identification"),
    Prediction(rationale="a person who is migrating from one country to another", answer="personality migration"),
    Prediction(rationale="bad friend", answer="fake personality"),
]

multichain = MultiChainComparison("question -> answer")

response = multichain(question="distinguishes an individual from others by describing feelings, thinking, and behaviors", completions=completions)

print(response)

Prediction(
    rationale='correct the answers. Personality distinguishes an individual from others by describing feelings, 
thinking, and behaviors. It is the unique pattern of characteristics that defines an individual.',
    answer='Personality'
)

#### ReAct (reasoning and acting)


In [108]:
import dspy
from dspy import ReAct, Signature
from rich import print

# Define a simple signature for basic question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA)

# Call the ReAct module on a particular input
question = 'Aside from the Apple Remote, what other devices can control the program Apple Remote was originally designed to interact with?'
question = 'What is the real name of Batman?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

Question: What is the real name of Batman?

Final Predicted Answer (after ReAct process):

In [113]:
agent = dspy.ReAct("question -> answer")
agent(question="What's up!")

Prediction(
    observations=[],
    answer='Hey there! How can I help you today?'
)

In [114]:
agent(question="Could you help me with some basic math?")

Prediction(
    observations=['Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.'],
    answer=''
)

### Data & Metrics

In [9]:
from dspy import Example

ex1 = Example(question="What's position are open now?", answer="Agent Call Center", hint="Consult data base for updates on position jobs.", table_ref="hr_jobs")

print(ex1)

Example({'question': 'Whats position are open now?', 'answer': 'Agent Call Center', 'hint': 'Consult data base for 
updates on position jobs.', 'table_ref': 'hr_jobs'}) (input_keys=None)

In [10]:
ex2 = Example(question="According to company rules, what is the most valuable employee?", answer="Sales Agent").with_inputs("company_rules")
print(ex2)

Example({'question': 'According to company rules, what is the most valuable employee?', 'answer': 'Sales Agent'}) 
(input_keys={'company_rules'})

In [16]:
ex2.inputs(), ex2.labels()

(Example({}) (input_keys=None),
 Example({'question': 'According to company rules, what is the most valuable employee?', 'answer': 'Sales Agent'}) (input_keys=None))

#### Metrics

### Optimizers